### 1. 模块加载


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import jieba
import glob
import copy
import time

from keras.models import Model
from keras.layers import multiply
from keras.layers import Dense, Embedding, Input, Flatten
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint,TensorBoard
from keras.layers.wrappers import TimeDistributed


import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

Using TensorFlow backend.


### 2. 目录定义查看


In [2]:
from subprocess import check_output
path = '../nocode/'
inp = 'input/testproject/'
oup = 'output/testproject/'
logp = 'logs/testproject/'
modp = 'models/testproject/'

# print(check_output(["ls", path + inp]).decode("utf8"))

EMBEDDING_FILE = path + 'wordvector/wiki.zh.vec'
EMBEDDING_FILE = path + 'wordvector/crawl-300d-2M.vec'

# 训练数据总集
data_all_file = path + inp + 'sematic_label_train.csv'

# 训练数据集
train_file = path + inp + 'train.csv'
# 测试数据集
# test_file = path + inp + 'test_alll.csv'
test_file = path + inp + 'test.csv'

TXT_DATA_FILE = path + inp + 'sematic_train.txt'
XLSX_DATA_FILE = path + inp + 'sematic_train.xlsx'
CSV_DATA_FILE = path + inp + 'sematic_train.csv'

# 结果文件
res_file = path + inp + 'baseline.csv'
# 模型文件
model_path = path + modp + 'model_t.h5'
out_path = path + oup + 'baseline.csv'
tensor_path = path + logp + 'baseline.csv'

# model_path = './model/model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'


### 4. 数据集 训练转化

In [3]:
# translate to 1 hot
# 去非nan
data_all = pd.read_csv(data_all_file)
data_all = data_all[~(data_all["评论内容"].isnull())]
# list_sentences_test = test["评论内容"].fillna("CVxTz").values

# 变成one hot
data_all['postive']=0
data_all['neutral']=0
data_all['negative']=0
data_all.loc[data_all['分类'] == 1, 'postive'] = 1
data_all.loc[data_all['分类'] == 0, 'neutral'] = 1
data_all.loc[data_all['分类'] == -1, 'negative'] = 1

# # 拆分测试集,训练时跳过  *******************************
# train_all = data_all.sample(frac=1).reset_index(drop=True)
# lenth_train = train_all.shape[0]
# spint = int(1*lenth_train)
# train = train_all.loc[0:spint,:]
# test = train_all.loc[spint:,:]
# train.to_csv(train_file, index=False)
# test.to_csv(test_file, index=False)
# # 拆分测试集,训练时跳过  *******************************


### 5. 开始训练

In [4]:
# max_features = 999999
max_features = 200000
maxlen = 300
memn = 100
dropn = 0.5

# train_file = path + inp + 'train.csv'
train = pd.read_csv(train_file)
print(train.shape)
# test_file = path + inp + 'test_alll.csv'
test = pd.read_csv(test_file)
print(test.shape)
for i1 in train.index:
    train.loc[i1, "评论内容"]=" ".join(jieba.cut(train.loc[i1, "评论内容"]))
for i1 in test.index:
    test.loc[i1, "评论内容"]=" ".join(jieba.cut(test.loc[i1, "评论内容"]))
    
list_sentences_train = train["评论内容"].fillna("CVxTz").values
list_sentences_test = test["评论内容"].fillna("CVxTz").values
# list_classes = [i1 for i1 in train.columns]
list_classes = ["negative","neutral","postive"]
try:
    list_classes.remove("评论内容")
    list_classes.remove("id")
except Exception as e:
    pass

y = train[list_classes].values

# list_sentences_test = test["评论内容"].fillna("CVxTz").values

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


(737, 7)
(184, 7)


Loading model cost 0.687 seconds.
Prefix dict has been built succesfully.


In [5]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)


In [6]:
# first way attention
# def attention_3d_block(inputs):
#     #input_dim = int(inputs.shape[2])
# #     a = Permute((2, 1))(inputs)
#     x = Reshape((maxlen * ,), name='reshape_1')(x)
#     a_probs = Dense(maxlen, activation='softmax')(inputs)
# #     a_probs = Permute((2, 1), name='attention_vec')(a)
#     #output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
#     output_attention_mul = multiply([inputs, a_probs], name='attention_mul')
#     return output_attention_mul

# lstm_out = Bidirectional(LSTM(lstm_units, return_sequences=True), name='bilstm')(drop1)
# attention_mul = attention_3d_block(lstm_out)
# attention_flatten = Flatten()(attention_mul)
# drop2 = Dropout(0.3)(attention_flatten)

def get_model(memn,dropn):
    embed_size = 105
    embed_size = 300
    # 时间步 = maxlen
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    # memory units = 50
    x = Bidirectional(LSTM(memn, return_sequences=True))(x)
# #     x = GlobalMaxPool1D()(x)
# #     x = Reshape((maxlen * memn,), name='reshape_1')(x)
#     a_probs = Dense(maxlen, activation='softmax')(x)
# #     a_probs = Permute((2, 1), name='attention_vec')(a)
#     #output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
#     x = multiply([x, a_probs], name='attention_mul')

# #     x = attention_3d_block(x)
#     x = Dropout(dropn)(x)
#     x = Bidirectional(LSTM(memn, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(dropn)(x)
    x = Dense(memn, activation="relu")(x)
    x = Dropout(dropn)(x)
    x = Dense(3, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [7]:
batch_size = 32
epochs = 100

checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorb = TensorBoard(log_dir=tensor_path, histogram_freq=10, write_graph=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
callbacks_list = [checkpoint, early, tensorb] #early

# model = get_model(200, 0.2)
model = get_model(memn, dropn)
print(model.summary())
# mlist = list(range(1,7))
# droplist = list(range(1,7))
# for memn in mlist:
#     for dropn in droplist:
#         model = get_model(memn*50, dropn*0.1)
#         start = time.time()
#         model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=callbacks_list)
#         end = time.time()
#         print(end-start)
#         print(memn*50, dropn*0.1)
#         print("*".center(100,"#"))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          60000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300, 200)          320800    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
__________

In [8]:
start = time.time()
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=callbacks_list)
end = time.time()
print(end-start)

Train on 589 samples, validate on 148 samples
Epoch 1/100
589/589 [==============================] - 18s 30ms/step - loss: 0.6049 - acc: 0.6333 - val_loss: 0.6253 - val_acc: 0.6937

Epoch 00001: val_loss improved from inf to 0.62535, saving model to ../nocode/models/testproject/model_t.h5
Epoch 2/100
589/589 [==============================] - 16s 28ms/step - loss: 0.5455 - acc: 0.6814 - val_loss: 0.5891 - val_acc: 0.7027

Epoch 00002: val_loss improved from 0.62535 to 0.58914, saving model to ../nocode/models/testproject/model_t.h5
Epoch 3/100
589/589 [==============================] - 17s 28ms/step - loss: 0.5309 - acc: 0.7063 - val_loss: 0.5704 - val_acc: 0.7590

Epoch 00003: val_loss improved from 0.58914 to 0.57036, saving model to ../nocode/models/testproject/model_t.h5
Epoch 4/100
589/589 [==============================] - 16s 28ms/step - loss: 0.4262 - acc: 0.8387 - val_loss: 0.4995 - val_acc: 0.7703

Epoch 00004: val_loss improved from 0.57036 to 0.49947, saving model to ../noc

In [9]:
model.load_weights(model_path)

y_test = model.predict(X_te)
# predict(self, x, batch_size=32, verbose=0)
# predict_classes(self, x, batch_size=32, verbose=1)
# predict_proba(self, x, batch_size=32, verbose=1)
# evaluate(self, x, y, batch_size=32, verbose=1, sample_weight=None)

# test_file = path + inp + 'test_alll.csv'
sample_submission = pd.read_csv(test_file)
sample_submission["分类"]=0
sample_submission["postive"]=0
sample_submission["neutral"]=0
sample_submission["negative"]=0
print(train.head())
print(sample_submission.head())
sample_submission[list_classes] = y_test
sample_submission["max"]=sample_submission[list_classes].max(axis=1)

for indexs in sample_submission.index:  
    for  i2 in list_classes:  
        if(sample_submission.loc[indexs,i2] ==sample_submission.loc[indexs,"max"]):
            sample_submission.loc[indexs,"predict"]=i2
for i1 in list_classes:
    sample_submission.rename(columns={i1: "pred_" + i1}, inplace=True)
sample_submission.to_csv(res_file, index=False)

                                                评论内容   评论数    点赞数  分类  \
0                                  肖杰   实力派   有 资本 狂   1.0   73.0   1   
1                                 鹿晗 鹿晗 ， 一鹿 伴晗 ， 加油  26.0  174.0   1   
2    冯 老板 的 这段 舞 真的 精彩 ， 那个 慢动作 太帅 了 ， 看 了 N 遍 了 ， 赞   3.0   36.0   1   
3                        感觉 有 黑幕 ， 陈伟霆队 跳 的 更好 ， ， ，   0.0    7.0  -1   
4  白羊座 本来 就 很 冲动 一点 的 ， 所以 请 任何人 不要 这样 说 鹿晗 ， 换个 ...   0.0    3.0  -1   

   postive  neutral  negative  
0        1        0         0  
1        1        0         0  
2        1        0         0  
3        0        0         1  
4        0        0         1  
                                                评论内容   评论数    点赞数  分类  \
0                                        鹿晗第一，鹿晗舞蹈精灵  14.0  194.0   0   
1  卜凡好帅好可爱，学的好认真，虽然偶像练习生最终没有选你出道，那不是你的问题，只是名额有限，不...   0.0   31.0   0   
2  钢王队错了？最少，大饼错了。很多人说不出错在什麽地方，让我来解释一下。「蝴蝶效应」不但是指一...  71.0   35.0   0   
3                                       什么破节目组，水都不给喝   1.0   15.0   0   
4   

In [11]:
# 正确率评估
score = model.evaluate(X_t, y, batch_size=batch_size)
print(score)
test_pd = pd.read_csv(test_file)
res_pd = pd.read_csv(res_file)
total_pd=pd.concat([res_pd,test_pd], join='outer', axis=1)
total_right=0
total_num=0
for i1 in list_classes:
    tmp_obj=total_pd[total_pd[i1] == 1]
    sum_num=tmp_obj.shape[0]
    right_num=tmp_obj[tmp_obj["predict"] == i1].shape[0]
    total_right += right_num
    total_num += sum_num
    try:
        print("%s, sum_num: %d, right_num: %d, accuracy: %.3f" % (i1,sum_num,right_num,right_num/sum_num))
    except Exception as e:
        print("%s, sum_num: %d, right_num: %d, error: %s" % (i1,sum_num,right_num,str(e)))
        
print("total data, total_num: %d, total_right: %d, accuracy: %.3f" % (total_num,total_right,total_right/total_num))

737/737 [==============================] - 2s 2ms/step
[0.2558635689692226, 0.9081863470608053]
negative, sum_num: 88, right_num: 80, accuracy: 0.909
neutral, sum_num: 13, right_num: 0, accuracy: 0.000
postive, sum_num: 83, right_num: 70, accuracy: 0.843
total data, total_num: 184, total_right: 150, accuracy: 0.815
